In [8]:
import glob as gl
import pandas as pd
import numpy as np
import time
import dask
import math as math
import datetime as dt

In [9]:
csvlist = gl.glob("*.csv")

In [10]:
len(csvlist)

1744

In [11]:
def listshift(listofcsvs):
    newcsvlist = [listofcsvs[-1]] + listofcsvs[:-1]
    return newcsvlist

In [12]:
csvlist2 = listshift(csvlist)

In [13]:
csvtestlist = csvlist[0:50]
csvtestlist2 = csvlist2[0:50]

In [14]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=8, n_workers=1)
client

Client Scheduler: tcp://127.0.0.1:62967 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 8 Memory: 17.18 GB


In [15]:
def cointtest(stock1, stock2, lagnumber, optionals=0):
    csv1 = pd.read_csv(stock1)
    price1_list = csv1['contract_best_buy_yes_price'].to_numpy()
    price1 = price1_list[0:-lagnumber]
    csv2 = pd.read_csv(stock2)
    price2_list = csv2['contract_best_buy_yes_price'].to_numpy()
    avg_list = []
    m_list = []
    c_list = []
    ratiolistlist = []
    goodlist = []
    if len(price1_list) == len(price2_list):
        try:
            for x in np.arange(lagnumber):
                price2 = price2_list[x:(-lagnumber+x)]
                #print(price2)
                ratiolist = price1/price2
                avg = np.mean(ratiolist)
                x_axis = np.arange(len(ratiolist))/len(ratiolist)
                A = np.vstack([x_axis, np.ones(len(x_axis))]).T
                #print(avg)
                m, c = np.linalg.lstsq(A, ratiolist, rcond=None)[0]
                avg_list.append(avg)
                m_list.append(m)
                c_list.append(c)
                ratiolistlist.append(ratiolist)
                if m < 0.1 and m > -0.1:
                    goodlist.append([m, lagnumber])
                else:
                    None
        except:
            m_list.append(1)
    else:
        m_list.append(1)
    avg_slope = np.mean(m_list)
    del csv1
    del csv2
    if optionals == 1:
        liststuff1 = [stock1, stock2, goodlist, avg_slope, avg_list, m_list, c_list, ratiolistlist]
        return liststuff1
    else:
        del price1_list
        del price1
        del price2_list
        del avg_list
        del m_list
        del c_list
        del ratiolistlist
        liststuff2 = [stock1, stock2, goodlist, avg_slope]
        return liststuff2

In [16]:
def large_coint_test(stocklist1, stocklist2, lagnumber):
    goodstocklist = []
    badstocklist = []
    allstocklist = []
    dask_coint = dask.delayed(cointtest)
    dask_tasks_list = []
    for stock1, stock2 in zip(stocklist1, stocklist2):
        if stock1 == stock2:
            printphrase = str(stock1) + " and "+str(stock2)+" are the same stock, cannot do calculations!"
            allstocklist.append([printphrase])
            badstocklist.append([printphrase])
        else:
            task = dask_coint(stock1, stock2, lagnumber)
            dask_tasks_list.append(task)
    dask_persist_list = dask.persist(*dask_tasks_list)
    computations = dask.compute(dask_persist_list)
    return computations           

In [17]:
def tasktime(list1, list2, lagnumber):
    starttime = dt.datetime.now()
    taskfile = large_coint_test(list1, list2, lagnumber)
    endtime = dt.datetime.now()
    time_elapsed = (endtime-starttime)
    return taskfile, time_elapsed

In [22]:
wow, timeer = tasktime(csvlist, csvlist2, 120)

In [23]:
print(timeer)

0:02:41.018643


In [26]:
for x in np.arange(len(wow[0])):
    if wow[0][x][2] != []:
        print(wow[0][x][0], wow[0][x][1],wow[0][x][2][0])
    else:
        None

Who will win the 2020 Pennsylvania Democratic primary?Tom Steyer.csv Who will win the 2020 Minnesota Democratic primary?Andrew Yang.csv [0.05393159947327541, 120]
Who will win the 2020 Utah Democratic primary?Julián Castro.csv Will Howard Schultz run for president in 2020?Will Howard Schultz run for president in 2020?.csv [-0.006840479771887559, 120]
Who will win the 2020 Connecticut Democratic primary?Andrew Yang.csv Who will win the 2020 Louisiana Democratic primary?Joe Biden.csv [-0.03298491638442237, 120]
Who will win the 2020 Virginia Democratic primary?Cory Booker.csv Who will win the 2020 Connecticut Democratic primary?Andrew Yang.csv [0.014601102695537544, 120]
Who will win the 2020 Oklahoma Democratic primary?Tom Steyer.csv Who will win the 2020 Virginia Democratic primary?Cory Booker.csv [-4.264120184082122e-15, 120]
What will Andrew Yang's RCP average support be at the end of November?1.9% or lower.csv How many Democrats will have dropped out by Nov. 30?9.csv [0.020224556729

['Who will win the 2020 Nevada Democratic caucuses?Hillary Clinton.csv',
 'Who will win the 2020 Democratic presidential nomination?Eric Swalwell.csv',
 [],
 0.1428520104013957]